In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import TruncatedSVD


In [135]:
# Loading datasets
ratings = pd.read_csv("../data/ratings.csv")  # userId, movieId, rating, timestamp
movies = pd.read_csv("../data/movies.csv")  # movieId, title

# Create a dictionary mapping movieId to title
movie_id_to_title = dict(zip(movies['movieId'], movies['title']))


## Step 1: Preprocessing Block

In [3]:
def preprocess_ratings(ratings, movies):
    """
    Preprocess the ratings data to create user-item matrix and ID mappings.
    
    :param ratings: DataFrame with user-item ratings
    :param movies: DataFrame with movie data
    
    :return: A tuple containing the user-item matrix and ID mappings
    """
    # Movie ID to index map
    movie_id_to_index = {movie_id: index for index, movie_id in enumerate(movies['movieId'])}
    index_to_movie_id = {index: movie_id for movie_id, index in movie_id_to_index.items()}
    
    # User ID to index map
    user_id_to_index = {user_id: index for index, user_id in enumerate(ratings['userId'].unique())}
    index_to_user_id = {index: user_id for user_id, index in user_id_to_index.items()}
    
    # Map movie and user IDs to indices
    ratings['movieIndex'] = ratings['movieId'].map(movie_id_to_index)
    ratings['userIndex'] = ratings['userId'].map(user_id_to_index)
    
    # Create the user-item matrix
    user_movie_matrix = ratings.pivot(index='userIndex', columns='movieIndex', values='rating').fillna(0)
    
    return user_movie_matrix, movie_id_to_index, index_to_movie_id, user_id_to_index, index_to_user_id


In [4]:
user_movie_matrix, movie_id_to_index, index_to_movie_id, user_id_to_index, index_to_user_id = preprocess_ratings(ratings=ratings,movies=movies)

## Step 2: Recommendation Functions 

#### 1. SVD Recommendation

In [5]:
def svd_recommendation(user_movie_matrix,  n_components=50):
    """
    Apply SVD for matrix factorization using preprocessed data.
    :param user_movie_matrix: The preprocessed user-item matrix
    :param n_components: Number of latent features to learn
    
    :return: A matrix of predictions (user-item)
    """
    # Apply SVD
    svd = TruncatedSVD(n_components=n_components)
    latent_matrix = svd.fit_transform(user_movie_matrix)
    
    # Get predictions
    prediction_matrix = np.dot(latent_matrix, svd.components_)
    
    return prediction_matrix


In [6]:
user_movie_matrix

movieIndex,0,1,2,3,4,5,6,7,8,9,...,9732,9733,9734,9735,9736,9737,9738,9739,9740,9741
userIndex,,,,,,,,,,,,,,,,,,,,,
0,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
606,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
svd_prediction_matrix=svd_recommendation(user_movie_matrix,200)
pd.DataFrame(svd_prediction_matrix)

,0,1,2,3,4,5,6,7,8,9,...,9714,9715,9716,9717,9718,9719,9720,9721,9722,9723
0,4.672979,-0.321925,4.112398,0.086995,0.198575,3.773911,0.606676,-0.060055,-0.142912,-0.157497,...,-0.004407,-0.003777,-0.005036,-0.005036,-0.004407,-0.005036,-0.004407,-0.004407,-0.004407,-0.023967
1,-0.108730,0.266511,-0.016596,-0.014251,0.134605,-0.020804,-0.167901,0.020457,-0.125315,0.246006,...,0.067011,0.057438,0.076584,0.076584,0.067011,0.076584,0.067011,0.067011,0.067011,0.078015
2,-0.011645,0.089615,0.028654,0.022930,0.069516,0.274509,0.062688,0.002676,0.028313,0.003970,...,0.002846,0.002439,0.003252,0.003252,0.002846,0.003252,0.002846,0.002846,0.002846,-0.006081
3,-0.682004,-0.480568,-0.132291,-0.002483,-0.170560,-0.380540,0.593426,-0.117567,-0.245097,-0.177800,...,0.071016,0.060871,0.081161,0.081161,0.071016,0.081161,0.071016,0.071016,0.071016,-0.004277
4,2.418371,0.381345,0.114885,-0.003391,0.135133,-0.080136,0.480529,0.213242,-0.224749,0.154475,...,0.038306,0.032834,0.043778,0.043778,0.038306,0.043778,0.038306,0.038306,0.038306,0.072769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,2.457690,-0.005732,0.069362,0.012574,0.024134,-0.138875,2.573247,0.007576,0.002918,0.073086,...,-0.013939,-0.011948,-0.015930,-0.015930,-0.013939,-0.015930,-0.013939,-0.013939,-0.013939,-0.022782
606,4.265077,0.005149,-0.154587,0.332863,0.225641,-0.549431,0.502324,0.181124,-0.183952,0.598693,...,-0.134520,-0.115303,-0.153737,-0.153737,-0.134520,-0.153737,-0.134520,-0.134520,-0.134520,-0.023087
607,2.274265,2.031667,2.042427,-0.013484,-0.094525,-0.258147,0.035141,-0.047308,0.086961,4.091352,...,-0.009706,-0.008320,-0.011093,-0.011093,-0.009706,-0.011093,-0.009706,-0.009706,-0.009706,0.018687
608,1.956206,0.171280,0.294059,-0.095678,0.182499,-0.105733,0.274492,-0.002127,0.131904,1.655205,...,-0.011306,-0.009691,-0.012921,-0.012921,-0.011306,-0.012921,-0.011306,-0.011306,-0.011306,0.042694


#### 2. User-User Collaborative Filtering

In [20]:
def user_user_collaborative_filtering( user_movie_matrix,  k=5):
    """
    Recommend movies based on user-user similarity (k-nearest neighbors) using preprocessed data.
    
    :param user_movie_matrix: The preprocessed user-item matrix
    :param k: The number of nearest neighbors to consider
    
    :return: A dictionary of userId -> recommended movie IDs
    """
    # Compute the user-user similarity using k-NN
    knn = NearestNeighbors(metric='cosine', algorithm='brute')
    knn.fit(user_movie_matrix)
    
    recommendations = {}
    
    for user_id in user_movie_matrix.index:
        distances, indices = knn.kneighbors(user_movie_matrix.loc[user_id].values.reshape(1, -1), n_neighbors=k)
        
        # Get movies rated highly by similar users
        similar_users = user_movie_matrix.iloc[indices[0]]
        recommended_movie_indices = similar_users.mean(axis=0).sort_values(ascending=False).index[:10]
        
        # Convert movie indices back to movie IDs
        recommended_movie_ids = [index_to_movie_id[idx] for idx in recommended_movie_indices]
        recommendations[index_to_user_id[user_id]] = recommended_movie_ids
    
    return recommendations


In [21]:
user_user_recommendations = user_user_collaborative_filtering(user_movie_matrix,k=10)

In [22]:
user_user_recommendations

{np.int64(1): [1198, 1197, 2571, 1196, 608, 260, 1136, 1291, 1200, 2947],
 np.int64(2): [58559,
  79132,
  318,
  106782,
  3578,
  48516,
  356,
  2571,
  109487,
  74458],
 np.int64(3): [1214, 1200, 1371, 2288, 3703, 32, 1196, 480, 2028, 2858],
 np.int64(4): [1265, 2858, 912, 904, 858, 1136, 1197, 608, 2997, 1206],
 np.int64(5): [457, 296, 527, 590, 356, 150, 318, 589, 380, 364],
 np.int64(6): [356, 480, 457, 500, 588, 318, 110, 364, 150, 377],
 np.int64(7): [356, 260, 1196, 1210, 4306, 8961, 2571, 5952, 7153, 2762],
 np.int64(8): [590, 356, 457, 34, 377, 150, 380, 318, 364, 480],
 np.int64(9): [4993, 1198, 5952, 2571, 7153, 1196, 1270, 1210, 260, 1291],
 np.int64(10): [356, 68954, 81845, 4306, 45720, 6942, 597, 31685, 6377, 4447],
 np.int64(11): [457, 110, 150, 318, 349, 356, 589, 380, 292, 161],
 np.int64(12): [356, 39, 1721, 2572, 6942, 357, 1380, 4246, 838, 1265],
 np.int64(13): [3578, 2571, 1198, 3996, 1210, 260, 1196, 3793, 4011, 4993],
 np.int64(14): [590, 296, 318, 356, 527, 

#### 3. Item-Item Collaborative Filtering

In [23]:
def item_item_collaborative_filtering(user_movie_matrix, k=5):
    """
    Recommend movies based on item-item similarity (k-nearest neighbors) using preprocessed data.
    
    :param user_movie_matrix: The preprocessed user-item matrix
    :param k: The number of nearest neighbors to consider
    
    :return: A dictionary of movieId -> recommended movie IDs
    """
    # Compute the item-item similarity using k-NN
    knn = NearestNeighbors(metric='cosine', algorithm='brute')
    knn.fit(user_movie_matrix.T)  # Transpose to make movies as rows
    
    recommendations = {}
    
    for movie_id in user_movie_matrix.columns:
        distances, indices = knn.kneighbors(user_movie_matrix.T.loc[movie_id].values.reshape(1, -1), n_neighbors=k)
        
        # Get similar movies based on the user ratings
        similar_movie_indices = user_movie_matrix.columns[indices[0]]
        
        # Convert movie indices back to movie IDs
        similar_movie_ids = [index_to_movie_id[idx] for idx in similar_movie_indices]
        recommendations[index_to_movie_id[movie_id]] = similar_movie_ids
    
    return recommendations


In [24]:
item_tem_recommendations=item_item_collaborative_filtering(user_movie_matrix,k=10)

In [25]:
item_tem_recommendations 

{1: [1, 3114, 480, 780, 260, 356, 364, 1210, 648, 1265],
 2: [2, 364, 500, 367, 480, 586, 551, 588, 595, 19],
 3: [3, 3450, 762, 788, 736, 5, 95, 65, 708, 7],
 4: [4, 113, 987, 55, 722, 726, 775, 979, 189, 336],
 5: [5, 7, 79, 762, 62, 3, 277, 637, 736, 1073],
 6: [6, 733, 32, 293, 16, 608, 457, 47, 95, 163],
 7: [7, 708, 5, 62, 141, 852, 3, 648, 17, 11],
 8: [8, 271, 174, 502, 217, 542, 484, 259, 250, 374],
 9: [9, 634, 786, 74, 667, 879, 79, 743, 737, 656],
 10: [10, 165, 380, 349, 377, 592, 316, 480, 153, 648],
 11: [11, 597, 440, 539, 587, 457, 357, 590, 380, 21],
 12: [12, 39400, 6263, 6379, 27311, 79274, 66783, 92048, 3511, 1646],
 13: [13, 1489, 1919, 1547, 6358, 3410, 4783, 4449, 709, 54],
 14: [14, 52, 36, 25, 628, 100, 1366, 805, 116, 17],
 15: [15, 93, 340, 415, 795, 491, 276, 291, 54, 437],
 16: [16, 1213, 6, 4262, 555, 1466, 431, 1089, 50, 1222],
 17: [17, 36, 838, 141, 224, 357, 25, 708, 21, 2396],
 18: [18, 5298, 2691, 3992, 5628, 4849, 6947, 5423, 7354, 4313],
 19: [19,

## Revised Scenarios
#### 1. New User (No History)

In [34]:
def new_user_recommendation_with_diversity(
    ratings, 
    item_item_recommendations, 
    watched_movie_id=None, 
    top_n=10, 
    diversity_factor=0.3
):
    """
    Recommends movies for a new user with a focus on diversity.
    - If no movie is watched, recommend a mix of popular and random movies.
    - If a movie is watched, recommend a mix of similar and diverse movies.
    
    :param ratings: DataFrame with columns ['userId', 'movieId', 'rating']
    :param item_item_recommendations: Precomputed dictionary of movieId -> recommended movie IDs
    :param watched_movie_id: ID of the movie watched by the new user
    :param top_n: Number of movies to recommend
    :param diversity_factor: Proportion of recommendations to include as random or diverse
    
    :return: List of recommended movie IDs
    """
    if watched_movie_id is None:
        # Popular movies
        popular_movies = (
            ratings.groupby('movieId')['rating']
            .mean()
            .sort_values(ascending=False)
            .head(top_n)
            .index.tolist()
        )
        
        # Add diversity by mixing in some random movies
        all_movies = ratings['movieId'].unique()
        num_diverse = int(top_n * diversity_factor)
        random_movies = list(np.random.choice(all_movies, size=num_diverse, replace=False))
        
        return popular_movies[:top_n - num_diverse] + random_movies
    
    else:
        # Similar movies based on item-item recommendations
        if watched_movie_id in item_item_recommendations:
            similar_movies = item_item_recommendations[watched_movie_id][:top_n]
            print("get similar movie... ")
        else:
            print("no movie id!")
            similar_movies = []
        
        # Add diversity by mixing in random movies
        all_movies = ratings['movieId'].unique()
        num_diverse = int(top_n * diversity_factor)
        random_movies = list(np.random.choice(all_movies, size=num_diverse, replace=False))
        
        return similar_movies[:top_n - num_diverse] + random_movies


In [37]:
new_user_recommendation_with_diversity(ratings,item_tem_recommendations)

[187717,
 6983,
 5328,
 95843,
 3941,
 3940,
 3939,
 np.int64(36708),
 np.int64(108932),
 np.int64(2996)]

## 2. Returning User (History Exists)
#### Home Page Recommendations

In [82]:
import random

def home_page_recommendations_with_diversity(
    user_id, 
    ratings, 
    user_movie_matrix, 
    item_item_recommendations, 
    user_user_recommendations, 
    svd_prediction_matrix, 
    weights=(0.4, 0.3, 0.3), 
    top_n=50, 
    global_top_movies=5  # Number of global top rated movies to include
):
    """
    Generates home page recommendations for a returning user based on their history, 
    with randomization, diversity, and sorting by predicted and average ratings.
    
    :param user_id: ID of the user for whom to generate recommendations
    :param ratings: DataFrame with columns ['userId', 'movieId', 'rating']
    :param user_movie_matrix: User-item matrix
    :param item_item_recommendations: Precomputed item-item recommendations
    :param user_user_recommendations: Precomputed user-user recommendations
    :param svd_prediction_matrix: SVD-based prediction matrix
    :param weights: Weights for blending the recommendation methods (item-item, user-user, SVD)
    :param top_n: Number of movies to recommend
    :param diversity_factor: Factor to control diversity in the recommendations (between 0 and 1)
    :param global_top_movies: Number of global top rated movies to include
    :return: List of recommended movie IDs
    """
    
    # Get the user's ratings history
    user_ratings = ratings[ratings['userId'] == user_id]
    rated_movies = user_ratings['movieId'].tolist()
    
    # ----- 1. Item-Item Collaborative Filtering -----
    top_rated_movies = user_ratings.sort_values(by='rating', ascending=False)['movieId'].head(5).tolist()
    item_item_movies = []
    for movie_id in top_rated_movies:
        if movie_id in item_item_recommendations:
            item_item_movies.extend(item_item_recommendations[movie_id])
    
    item_item_movies = [m for m in item_item_movies if m not in rated_movies]
    
    # ----- 2. User-User Collaborative Filtering -----
    user_user_movies = []
    if user_id in user_user_recommendations:
        similar_user_top_movies = user_user_recommendations[user_id]
        user_user_movies.extend([m for m in similar_user_top_movies if m not in rated_movies])
    
    # ----- 3. Predicted Ratings (SVD) -----
    user_index = user_id_to_index[user_id]
    if user_index in user_movie_matrix.index:
        svd_predictions = svd_prediction_matrix[user_index]
        svd_movie_ids = [index_to_movie_id[idx] for idx in user_movie_matrix.columns]
        svd_recommendations = sorted(
            zip(svd_movie_ids, svd_predictions), 
            key=lambda x: x[1], 
            reverse=True
        )
        svd_movies = [movie_id for movie_id, pred in svd_recommendations if movie_id not in rated_movies]
    else:
        svd_movies = []
    
    # ----- Global Top Rated Movies -----
    popular_movies = ratings.groupby('movieId')['rating'].mean().sort_values(ascending=False).head(global_top_movies).index.tolist()
    popular_movies = [m for m in popular_movies if m not in rated_movies]
    
    # ----- Blending Recommendations -----
    item_item_count = int(weights[0] * top_n)
    user_user_count = int(weights[1] * top_n)
    svd_count = top_n - item_item_count - user_user_count
    
    # Combine recommendations with weights
    combined_recommendations = (
        item_item_movies[:item_item_count] + 
        user_user_movies[:user_user_count] + 
        svd_movies[:svd_count]
    )
    
    # Add Global Top Rated Movies
    combined_recommendations = list(set(combined_recommendations) | set(popular_movies))
    
    # Introduce diversity by randomly shuffling the combined recommendations
    random.shuffle(combined_recommendations)
    
    # Get the top N recommended movies, considering the total size
    final_recommendations = combined_recommendations[:top_n]
    
    # ----- Get the predicted ratings for each movie in the final recommendations
    
    predicted_ratings = []
    for movie_id in final_recommendations:
        # Check if we have a predicted rating for the movie
        if movie_id in svd_movies:
            movie_index = movie_id_to_index[movie_id]
            predicted_rating = svd_prediction_matrix[user_index][movie_index]
        else:
            predicted_rating = 0  # Default to 0 if no prediction is available
        
        predicted_ratings.append((movie_id, predicted_rating))
    
    # Sort the recommendations by the predicted ratings
    final_recommendations = sorted(predicted_ratings, key=lambda x: x[1], reverse=True)
    
    # Return only the movie IDs sorted by their predicted ratings
    return [movie_id for movie_id, _ in final_recommendations[:top_n]]


In [88]:
# Call the function for a specific user (e.g., user 1)
recommended_movies = home_page_recommendations_with_diversity(
    user_id=1, 
    ratings=ratings, 
    user_movie_matrix=user_movie_matrix, 
    item_item_recommendations=item_tem_recommendations, 
    user_user_recommendations=user_user_recommendations, 
    svd_prediction_matrix=svd_prediction_matrix
)
recommended_movies

[1225,
 7,
 344,
 2194,
 104,
 180,
 588,
 1884,
 3213,
 380,
 1235,
 3941,
 2420,
 556,
 2110,
 364,
 95843,
 1939,
 2167,
 5328,
 187717,
 39234,
 2798,
 122904,
 1287,
 1449,
 8910,
 3745,
 1101,
 1250,
 2599,
 6983,
 165,
 1272,
 1200,
 292,
 293,
 858,
 153,
 318]

####  User Watches a New Movie

In [127]:
import random

def recommendations_after_new_movie(
    user_id, 
    new_movie_id, 
    ratings, 
    item_item_recommendations, 
    svd_prediction_matrix, 
    user_movie_matrix, 
    user_id_to_index, 
    index_to_movie_id, 
    weights=(0.7, 0.2), 
    top_n=10,
    rating_threshold=1
):
    """
    Generate recommendations after a user watches a new movie.
    
    :param user_id: ID of the user
    :param new_movie_id: ID of the newly watched movie
    :param ratings: DataFrame with columns ['userId', 'movieId', 'rating']
    :param item_item_recommendations: Precomputed item-item recommendations
    :param svd_prediction_matrix: SVD-based prediction matrix
    :param user_movie_matrix: User-item matrix
    :param user_id_to_index: Mapping from user ID to index in the SVD matrix
    :param index_to_movie_id: Mapping from matrix indices to movie IDs
    :param weights: Weights for blending the recommendation methods (item-item, SVD)
    :param top_n: Number of movies to recommend
    :param rating_threshold: Minimum predicted rating threshold for SVD-based recommendations
    :return: List of recommended movie IDs
    """
    # Get the user's ratings history
    user_rated_movies = ratings[ratings['userId'] == user_id]['movieId'].tolist()
    
    # ----- 1. Item-Item Collaborative Filtering -----
    similar_movies = []
    if new_movie_id in item_item_recommendations:
        similar_movies = item_item_recommendations[new_movie_id]
    
    # Remove movies the user has already rated
    similar_movies = [m for m in similar_movies if m not in user_rated_movies]
    
    # ----- 2. Predicted Ratings (SVD) -----
    svd_recommendations = []
    user_index = user_id_to_index[user_id]
    if user_index in user_movie_matrix.index:
        svd_predictions = svd_prediction_matrix[user_index]
        svd_movie_ids = [index_to_movie_id[idx] for idx in user_movie_matrix.columns]
        svd_recommendations = [
            movie_id for movie_id, pred in zip(svd_movie_ids, svd_predictions) 
            if pred > rating_threshold and movie_id not in user_rated_movies
        ]
    
    # ----- 3. Global Top Rated Movies -----
    global_top_movies = (
        ratings.groupby('movieId')['rating']
        .mean()
        .sort_values(ascending=False)
        .index.tolist()
    )
    
    # Remove movies the user has already rated
    global_top_movies = [m for m in global_top_movies if m not in user_rated_movies]
    
    # ----- Blending Recommendations -----
    item_item_count = int(weights[0] * top_n)
    svd_count = int(weights[1] * top_n)
    global_count = top_n - item_item_count - svd_count
    
    # Select the recommendations
    recommendations = (
        similar_movies[:item_item_count] +
        svd_recommendations[:svd_count] +
        global_top_movies[:global_count]
    )
    
    print(f"{len(similar_movies[:item_item_count])} / {item_item_count}")
    print(f"{len(svd_recommendations[:svd_count])} / {svd_count}")
    print(f"{len(global_top_movies[:global_count])} / {global_count}")
    
    # Deduplicate and introduce diversity
    recommendations = list(dict.fromkeys(recommendations))
    
    # Ensure the first 5 movies are strictly similar movies
    top_similar = similar_movies[:5]
    remaining_recommendations = [movie for movie in recommendations if movie not in top_similar]
    random.shuffle(remaining_recommendations)
    
    # Combine and limit to top_n
    final_recommendations = top_similar + remaining_recommendations[:top_n - len(top_similar)]
    
    # If not enough recommendations, fill with additional top-rated movies
    additional_movies = [m for m in global_top_movies if m not in final_recommendations]
    while len(final_recommendations) < top_n and additional_movies:
        final_recommendations.append(additional_movies.pop(0))
    
    return final_recommendations[:top_n]


In [137]:
recommendations = recommendations_after_new_movie(
    user_id=10,
    new_movie_id=20,
    ratings=ratings,
    item_item_recommendations=item_tem_recommendations,
    svd_prediction_matrix=svd_prediction_matrix,
    user_movie_matrix=user_movie_matrix,
    user_id_to_index=user_id_to_index,
    index_to_movie_id=index_to_movie_id,
    weights=(0.7, 0.2),
    top_n=20
)

print("Recommended Movies:", [movie_id_to_title[idx] for idx in recommendations ] )


10 / 14
4 / 4
2 / 2
Recommended Movies: ['Money Train (1995)', 'Ali G Indahouse (2002)', 'Cube Zero (2004)', 'Major League II (1994)', 'Savages (2012)', 'Haywire (2011)', 'Godfather, The (1972)', 'Jane Eyre (1944)', 'Legion (2010)', 'Ghostbusters (a.k.a. Ghost Busters) (1984)', 'Killers (2010)', 'Knights of Badassdom (2013)', 'Runner Runner (2013)', "Won't You Be My Neighbor? (2018)", 'Dangerous Minds (1995)', 'Groundhog Day (1993)', 'Rain (2001)', 'Goodbye Charlie (1964)', 'Sorority House Massacre (1986)', 'Slumber Party Massacre III (1990)']
